LSTM and GRU
============

IMDB dataset
------

In [5]:
from tensorflow.keras.datasets import imdb

(input_train, label_train), (input_test, label_test) = imdb.load_data(num_words=6000)
print(input_train[0]) # 단어 빈도수에 따른 정렬 예) the = 1
print(label_train[0]) # positive=1, negative=0

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [11]:
# 빈도 순위에 해당하는 단어 출력
word_index = imdb.get_word_index()
index_to_word={}
for key, value in word_index.items():
    index_to_word[value] = key

# 위 리뷰를 문장으로 출력
for word in input_train[0]:
    print(index_to_word[word], end = ' ')

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought and but of script you not while history he heart to real at barrel but when from one bit then have two

LSTM
---

In [12]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 6000 # 빈도순위 6000까지 가져오기
max_len =  130
embedding_size = 128

(input_train, label_train), (input_test, label_test) = imdb.load_data(num_words=max_features)

# pad_sequence : 데이터의 길이를 max_len으로 맞춰줌
# ex) X = np.array([[[0, 2], [1, 2, 3, 4], [3, 6, 7]]])
# pad_sequence(X)
# [[[0,0,0,2],
#   [1,2,3,4],
#   [0,3,6,7]]]
input_train = pad_sequences(input_train, maxlen=max_len)
input_test = pad_sequences(input_test, maxlen=max_len)

# return_sequences 인수를 True로 하면 출력 순서열 중 마지막 값만 출력하는 것이 아니라 
# 전체 순서열을 3차원 텐서 형태로 출력하므로 sequence-to-sequence 문제로 풀 수 있다.

# GlobalMaxPooling1D : 입력되는 벡터 중 가장 큰 값만 가져옴
def main():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, embedding_size),
    # tf.keras.layers.CuDNNLSTM(32, activation=tf.nn.tanh) -> 매우 빠름!!!!!!!
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,activation=tf.nn.relu, return_sequences=True)),
# GRU 사용법
#   tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, activation=tf.nn.tanh)),      
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(20, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(1, activation = tf.nn.relu), # positive, negative에 대한 예측이므로 Dense의 unit=1
    ])

# 활성화 함수
# relu : robust하지만 gradient vanishing문제를 잘 해결해줌, classification에 적합
# sigmoid : 결과값이 0~1을 가지기 때문에 gradient vanishing 문제가 발생하기 쉬우나 데이터를 보다 매끄럽게 표현 가능하여 regression에 적합
# tanh : sigmoid보다 gradient vanoshing 문제가 덜 함
# 위의 문제는 영화 감상자의 리뷰를 토대로 positivie, negative를 분류하는 문제이므로 활성화 함수로 relu가 적합!!!!!!

    model.compile(optimizer='adam', loss = 'mse', metrics=['mse'])

    history=model.fit(input_train, label_train, batch_size=128, epochs=1, verbose=1)
    
    return model

if __name__=='__main__':
    main()

Train on 25000 samples
25000/25000 [==============================] - 46s 2ms/sample - loss: 0.1809 - mse: 0.1809


Sentimental Analysis with CNN
----------------

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 6000
max_len =  130
embedding_size = 128

(input_train, label_train), (input_test, label_test) = imdb.load_data(num_words=max_features)
input_train = pad_sequences(input_train, maxlen=max_len)
input_test = pad_sequences(input_test, maxlen=max_len)


def conv(_input, node=16):
    # input값을 tf.float32 데이터 타입으로 바꿔줌
    _input = tf.cast(_input, tf.float32)
    output= tf.keras.layers.concatenate([
        tf.keras.layers.Conv1D(filters=node, padding='same', strides=1, kernel_size=1)(_input),
        tf.keras.layers.Conv1D(filters=node, padding='same', strides=1, kernel_size=2)(_input),
        tf.keras.layers.Conv1D(filters=node, padding='same', strides=1, kernel_size=3)(_input),
    ])

    return output

def expand_dim(_input):
    return tf.expand_dims(_input, -1)
    
    
def main():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, embedding_size),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(10, return_sequences=True)),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Lambda(expand_dim),
    tf.keras.layers.Lambda(conv),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid, dtype=tf.float32),
    ])

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['acc'])

    history = model.fit(input_train, label_train, validation_split=.2, batch_size=100, epochs=5)

if __name__=='__main__':
    main()

W0808 15:33:53.995616 21068 deprecation.py:323] From c:\users\default.desktop-s5q9gaa\anaconda3\envs\yangjaehub\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 32s 2ms/sample - loss: 0.4520 - acc: 0.7752 - val_loss: 0.3197 - val_acc: 0.8610
Epoch 2/5
20000/20000 [==============================] - 29s 1ms/sample - loss: 0.2482 - acc: 0.8991 - val_loss: 0.3254 - val_acc: 0.8610
Epoch 3/5
20000/20000 [==============================] - 30s 1ms/sample - loss: 0.1724 - acc: 0.9337 - val_loss: 0.3528 - val_acc: 0.8552
Epoch 4/5
20000/20000 [==============================] - 29s 1ms/sample - loss: 0.1154 - acc: 0.9596 - val_loss: 0.4640 - val_acc: 0.8372
Epoch 5/5
20000/20000 [==============================] - 30s 1ms/sample - loss: 0.0736 - acc: 0.9754 - val_loss: 0.5530 - val_acc: 0.8412
